# 제주도 핫플레이스 - 인스타그램 크롤링

In [53]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [35]:
chromedriver = '/Workspace/02. DataAnalysis/05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. instagram 에 접속하여 로그인

In [36]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [37]:
with open('password.txt') as f:
    password = f.read(8)

In [18]:
len(password)

8

* email 과 password 의 xpath가 다르면 find_element_by_css_selector 로 찾으면 되지만 인스타의 경우 두개가 같아서 이름으로 찾기 find_element_by_name

In [38]:
email = 'jinyoun9.kim@gmail.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [39]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [41]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [49]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도 맛집 검색

In [46]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [47]:
# 먼저 3초로 설정하고 실행해본 후 몇초가 걸리는지 확인하고 숫자를 바꿔주는게 좋다 3에서 5로 변경
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글 열기

In [48]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

* 화면 너비에 따라 화면이 자유롭게 변하는 것 = 반응형(Responsive) 웹

### 4. 게시글 정보 가져오기

In [50]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [57]:
# 1) 본문 내용 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'협찬제주애월 썬셋맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주도 #비자림'

In [64]:
# 2) 본문 내용에서 해쉬테그 가져오기 '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월맛집',
 '#애월카페',
 '#애월읍맛집',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#제주도카페',
 '#제주카페',
 '#외도맛집',
 '#하귀맛집',
 '#한림맛집',
 '#한림카페',
 '#애월해안도로맛집',
 '#애월가볼만한곳',
 '#제주공항맛집',
 '#제주공항근처맛집',
 '#제주도',
 '#비자림']

In [67]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-17'

In [73]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9FW').text[4:-1]
except:
    like = 0
like

0

In [74]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0]
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [75]:
row = [content, date, like, place, tags]
row

['협찬제주애월 썬셋맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주도 #비자림',
 '2021-08-17',
 0,
 '',
 ['#애월맛집',
  '#애월카페',
  '#애월읍맛집',
  '#제주애월맛집',
  '#제주도맛집',
  '#제주맛집',
  '#제주도카페',
  '#제주카페',
  '#외도맛집',
  '#하귀맛집',
  '#한림맛집',
  '#한림카페',
  '#애월해안도로맛집',
  '#애월가볼만한곳',
  '#제주공항맛집',
  '#제주공항근처맛집',
  '#제주도',
  '#비자림']]

### 5. 다음 게시글로 이동

In [77]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [ ]:
#### 6을 하기 위해 팝업화면을 없애기(x 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 6. 여러개의 게시글 가져오기

In [96]:
from tqdm.notebook import tqdm

In [102]:
def get_content(driver):
    for i in tqdm(range(200)):
        #print(i, end=' ')
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
        
            content = soup.select_one('div.C4VMK > span').text
            content = unicodedata.normalize('NFC', content)
        except:
            content = ' '
        tags = re.findall(r'#[^\s#,\\]+', content)
        date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
        
        try:
            like = soup.select_one('div.Nm9FW').text[4:-1]
        except:
            like = 0
        
        try:
            place = soup.select('div.M30cS')[0]
            place = unicodedata.normalize('NFC', place)
        except:
            place = ''
        
        return [content, tags, date, like, place]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [113]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)


In [108]:
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(1)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)

In [109]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags
0,흑돈애에서 볼수 있는 화려한불쇼흑돼지와 해산물을 함께 먹을 수 있는 이곳 !육질좋은...,[],2021-08-19,0,
1,".💙 구좌 ""제주모랑"" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...","[#제주모랑, #제주소품샵, #세화소품샵, #제주기념품, #제주기념품샵, #구좌읍소...",2021-08-18,0,
2,(협찬)애월바다와 선셋🌅감성까지 더 한스페셜 카레맛집!😋다양한 카레와 토핑들로특별함...,"[#사이카레@saicurry_aewolOPEN, #사이카레, #구엄리맛집, #애월맛...",2021-08-18,0,
3,.#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...,"[#연동, #모니카옛날통닭간판부터, #제주맛집, #제주도맛집, #제주공항근처맛집, ...",2021-08-19,0,
4,".💙 구좌 ""제주모랑"" 💙사장님들이 손수 만든 수제소품들이 가득~좋은향이 가득한 구...","[#제주모랑, #제주소품샵, #세화소품샵, #제주기념품, #제주기념품샵, #구좌읍소...",2021-08-18,0,


In [118]:
df.tail()

,content,date,like,place,tags
176,여행의 마무리는 쇼핑이지ꗯ̤̮ 제주에서 찾은 새로운 리치향수ᰔ 넘흐 죠으다__이걸 ...,[],2021-08-19,0,
177,날씨요정 나야나🧚🏻‍♀️🤍,[],2021-08-19,0,
178,#어린왕자감귤밭 🍊 카페만 이용해도제주 어린왕자감귤밭에서 즐기는 이색데이트 코스알파...,"[#어린왕자감귤밭, #제주맛집, #제주핫플, #제주핫플레이스, #제주도가볼만한곳, ...",2021-08-19,0,
179,수비드 요리는 처음이라 설렌마음으로 간곳 고기가 너무 부드럽고 비트밥이랑 잘 어울렸...,"[#데일리, #어제, #요술식탁, #오랜만, #좋아요테러, #제주시레스토랑, #소통...",2021-08-19,0,
180,상효원을 방문해주신 분들의LOVELY한 후기 이미지를 모아보았습니다🌈💝소중한 추억 ...,[],2021-08-19,0,


In [119]:
df.to_csv('data/제주여행.csv', index=False)

In [117]:
for keyword in ['제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(10)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(2)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(3)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(3)

    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(5)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [120]:
driver.close()

### 4가지 CSV 파일의 중복을 제거한 후 통합 저장

In [124]:
jeju_df = pd.DataFrame([])
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    jeju_df = jeju_df.append(df)

jeju_df.head()

,content,date,like,place,tags
0,<광고>#제주분식 https://jejubunsig.modoo.at/ 📌제주시 귀아...,"['#제주분식', '#제주분식⠀간편하게', '#모당모당', '#돔베라면돔베김밥', ...",2021-08-16,0,NaN
1,오늘도 맛있고 두툼하고 부드럽게 준비 잘 해두었어요🙋‍♀️곧 시즌마감 예정인 애플망...,['#안녕하샌'],2021-08-19,0,NaN
2,제주도의 핫이슈!! 예약문의 폭주!! 애월 한라봉스시 스폐셜 메뉴 천국의계단(2-3...,"['#애월맛집', '#제주도맛집', '#제주맛집', '#애월스시', '#제주스시',...",2021-08-18,0,NaN
3,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,"['#연돈', '#제주도연돈', '#테이블링', '#등심까스', '#수제볼카츠', ...",2021-08-18,0,NaN
4,"협찬제주애월맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...","['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...",2021-08-18,0,NaN


In [125]:
jeju_df.tail()

,content,date,like,place,tags
176,여행의 마무리는 쇼핑이지ꗯ̤̮ 제주에서 찾은 새로운 리치향수ᰔ 넘흐 죠으다__이걸 ...,[],2021-08-19,0,NaN
177,날씨요정 나야나🧚🏻‍♀️🤍,[],2021-08-19,0,NaN
178,#어린왕자감귤밭 🍊 카페만 이용해도제주 어린왕자감귤밭에서 즐기는 이색데이트 코스알파...,"['#어린왕자감귤밭', '#제주맛집', '#제주핫플', '#제주핫플레이스', '#제...",2021-08-19,0,NaN
179,수비드 요리는 처음이라 설렌마음으로 간곳 고기가 너무 부드럽고 비트밥이랑 잘 어울렸...,"['#데일리', '#어제', '#요술식탁', '#오랜만', '#좋아요테러', '#제...",2021-08-19,0,NaN
180,상효원을 방문해주신 분들의LOVELY한 후기 이미지를 모아보았습니다🌈💝소중한 추억 ...,[],2021-08-19,0,NaN


In [ ]:
from wordcloud import wordcloud